In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [4]:
import numpy as np
import pandas as pd

In [5]:
butte = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Butte.csv")
calaveras = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Calaveras.csv")
dorado = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Dorado.csv")
fresno = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Fresno.csv")
humboldt = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Humboldt.csv")
kern = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_KERN.csv")
lassen = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Lassen.csv")
modoc = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Modoc.csv")
nevada = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Nevada.csv")
plumas = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Plumas.csv")
riverside = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Riverside.csv")
bernardino = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_San Bernardino.csv")
diego = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_San Diego.csv")
shasta = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Shasta.csv")
siskiyou = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Siskiyou.csv")

In [21]:

def append_dataframes(dataframes):
    if len(dataframes) == 0:
        return None
    elif len(dataframes) == 1:
        return dataframes[0]
    else:
        combined_df = pd.concat(dataframes, ignore_index=True)
        return combined_df


dataframes_list = [butte, calaveras, dorado, fresno, humboldt, kern, lassen, modoc, dorado, nevada, plumas, riverside, bernardino, diego, shasta, siskiyou]

fire_data = append_dataframes(dataframes_list)

In [6]:
butte.head(70)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,0,0,464,2.0,B,39.735278,-121.336389,6,20080621,20080831,6007.0,20080621,389,222,306,0,1.245353,96
1,1,1,465,1.0,B,39.533339,-121.530711,6,20010808,20010808,6007.0,20010808,411,228,322,0,2.611998,0
2,2,2,466,12.0,C,39.355089,-121.772239,6,20041012,20041012,6007.0,20041012,328,133,233,0,3.008472,0
3,3,3,467,0.1,A,39.790389,-121.515500,6,20041011,20041011,6007.0,20041011,322,133,228,0,5.034409,0
4,4,4,468,787.5,E,39.777800,-121.552470,6,20080621,20080729,6007.0,20080621,389,222,306,0,1.245353,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,65,65,529,0.5,B,39.703289,-121.529600,6,20050910,20050910,6007.0,20050910,389,222,306,0,4.138236,80
66,66,66,530,2.0,B,39.836600,-121.624400,6,20030705,20030705,6007.0,20030705,367,189,278,0,3.549707,0
67,67,67,531,2.0,B,39.490500,-121.323969,6,20020215,20020215,6007.0,20020215,139,72,106,3,2.758727,0
68,68,68,532,0.1,A,39.851500,-121.685050,6,19990505,19990505,6007.0,19990505,283,72,178,0,1.345358,0


In [7]:
calaveras.head(30)

,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,...,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,0,0,0,0,0,0,0,0,1.00,B,...,20070618,20070618,6009.0,20070618,184,74,135,0,1.513274,55.0
1,1,1,1,1,1,1,1,1,0.50,B,...,20001015,20001015,6009.0,20001015,179,-40,50,0,0.393192,0.0
2,2,2,2,2,2,2,2,2,0.10,A,...,20040825,20040825,6009.0,20040825,184,68,113,0,2.657545,71.0
3,3,3,3,3,3,3,3,3,1.00,B,...,20000825,20000825,6009.0,20000825,257,70,159,0,1.710815,0.0
4,4,4,4,4,4,4,4,4,0.10,A,...,20041008,20041008,6009.0,20041008,183,50,104,0,0.465721,71.0
5,5,5,5,5,5,5,5,5,0.10,A,...,20020329,20020329,6009.0,20020329,93,-81,16,0,0.649992,67.0
6,6,6,6,6,6,6,6,6,135.00,D,...,20060719,20060722,6009.0,20060719,209,126,167,25,2.668225,54.0
7,7,7,7,7,7,7,7,7,4.00,B,...,19990905,19990905,6009.0,19990905,209,51,119,0,0.478540,0.0
8,8,8,8,8,8,8,8,8,0.10,A,...,20000722,20000722,6009.0,20000722,271,59,184,0,2.093438,0.0
9,9,9,9,9,9,9,9,9,4.00,B,...,20060719,20060719,6009.0,20060719,209,126,167,25,2.668225,54.0


In [8]:
fresno.head(30)

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,0,0,0,3160,0.10,A,37.271389,-119.182222,6,20060722,20060723,6019.0,20060722,400,239,322,0,1.801363,99.500
1,1,1,1,3161,0.10,A,37.419167,-118.966389,6,20060626,20060627,6019.0,20060626,400,239,322,0,1.712835,150.000
2,2,2,2,3162,0.10,A,37.017075,-119.611833,6,20071214,20071214,6019.0,20071214,400,239,322,0,3.496543,88.000
3,3,3,3,3163,0.50,B,37.235833,-118.993056,6,20070806,20070807,6019.0,20070806,400,239,322,0,1.958288,71.000
4,4,4,4,3164,7456.00,G,36.903611,-118.801111,6,20080725,20081212,6019.0,20080725,400,239,322,0,1.767747,86.500
5,5,5,5,3165,0.20,A,37.444444,-119.120000,6,20060721,20060721,6019.0,20060721,400,239,322,0,2.289743,99.500
6,6,6,6,3166,0.01,A,37.110278,-119.114167,6,20070918,20070918,6019.0,20070918,400,239,322,0,1.627889,50.750
7,7,7,7,3167,0.33,B,36.795833,-118.898611,6,20060709,20060709,6019.0,20060709,400,239,322,0,1.509599,117.000
8,8,8,8,3168,0.10,A,36.086761,-120.119919,6,20010703,20010703,6019.0,20010703,433,256,344,0,2.391338,87.250
9,9,9,9,3169,0.10,A,37.086111,-119.216667,6,20080902,20080902,6019.0,20080902,400,239,322,0,1.224092,64.000


In [9]:
humboldt.head(3)

,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,...,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,0,0,0,0,0,3615,0.2,A,40.908611,-123.615278,...,20060917,20060917,6023.0,20060917,150,111,133,0,5.449113,103.0
1,1,1,1,1,1,3616,1.0,B,40.032119,-124.047669,...,20070401,20070401,6023.0,20070401,150,111,133,0,3.404772,139.0
2,2,2,2,2,2,3617,903.0,E,40.399400,-123.949400,...,20080621,20080718,6023.0,20080621,150,111,133,0,2.274308,62.0


In [10]:
kern.head(3)

,Unnamed: 0.1,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,0,4302,75.0,C,35.176956,-119.540608,6,20050804,20050804,6029.0,20050804,400,244,322,0,1.650767,64.5
1,1,4303,1223.0,F,35.305144,-118.666681,6,20030616,20030616,6029.0,20030616,361,200,283,0,1.646940,81.5
2,2,4304,80.0,C,34.921944,-119.404722,6,20080516,20080517,6029.0,20080516,400,244,322,0,2.450184,82.0


In [11]:
lassen.head(30)

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,0,0,0,5809,1.75,B,40.899167,-121.225278,6,20090802,20090802,6035.0,20090802,253,81,139,152,2.788690,109.0
1,1,1,1,5810,5.00,B,40.473200,-120.477700,6,20020723,20020724,6035.0,20020723,252,66,151,0,2.892818,141.0
2,2,2,2,5811,0.10,A,40.999722,-121.399722,6,20090802,20090805,6035.0,20090802,253,81,139,152,2.788690,109.0
3,3,3,3,5812,0.10,A,40.488056,-120.615778,6,20050908,20050908,6035.0,20050908,238,48,127,0,1.399039,117.0
4,4,4,4,5813,0.20,A,40.057500,-120.051700,6,20050909,20050909,6035.0,20050909,175,32,90,0,5.040073,117.0
5,5,5,5,5814,0.10,A,40.983200,-121.326100,6,20040814,20040814,6035.0,20040814,295,130,214,0,2.958949,147.0
6,6,6,6,5815,0.50,B,41.000300,-120.550300,6,20050909,20050910,6035.0,20050909,175,32,90,0,5.040073,117.0
7,7,7,7,5816,0.50,B,40.532861,-120.758119,6,20040524,20040524,6035.0,20040524,125,9,61,0,2.443862,173.0
8,8,8,8,5817,0.20,A,40.487400,-120.513800,6,20040814,20040815,6035.0,20040814,295,130,214,0,2.958949,147.0
9,9,9,9,5818,0.10,A,40.779900,-120.534400,6,20020718,20020718,6035.0,20020718,195,93,134,0,1.400147,141.0


In [12]:
modoc.head(30)

,Unnamed: 0.9,Unnamed: 0.8,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,...,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,0,0,0,0,0,0,0,0,0,8193,...,20070710,20070720,6049.0,20070710,348,120,233,0,1.891141,77.500
1,1,1,1,1,1,1,1,1,1,8194,...,20060814,20060815,6049.0,20060814,262,61,137,127,3.032787,98.000
2,2,2,2,2,2,2,2,2,2,8195,...,20070311,20070313,6049.0,20070311,144,13,63,0,2.077111,81.667
3,3,3,3,3,3,3,3,3,3,8196,...,20060814,20060814,6049.0,20060814,262,61,137,127,3.032787,98.000
4,4,4,4,4,4,4,4,4,4,8197,...,20070710,20070711,6049.0,20070710,348,120,233,0,1.891141,77.500
5,5,5,5,5,5,5,5,5,5,8198,...,20070929,20070929,6049.0,20070929,93,-24,34,51,1.570279,72.500
6,6,6,6,6,6,6,6,6,6,8199,...,20070710,20070716,6049.0,20070710,348,120,233,0,1.891141,77.500
7,7,7,7,7,7,7,7,7,7,8200,...,20060723,20060801,6049.0,20060723,322,129,226,0,1.922089,111.333
8,8,8,8,8,8,8,8,8,8,8201,...,20060724,20060725,6049.0,20060724,305,143,204,0,1.302462,111.333
9,9,9,9,9,9,9,9,9,9,8202,...,20070710,20070715,6049.0,20070710,348,120,233,0,1.891141,77.500


In [13]:
dorado.head(3)

,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,1793,0.1,A,38.649819,-120.938639,6,20020508,20020508,6017.0,20020508,144,-35,35,0,2.786203,172.0
1,1794,0.1,A,38.705000,-120.587222,6,20080604,20080604,6017.0,20080604,85,25,51,0,5.624106,70.0
2,1795,0.1,A,38.773411,-120.811150,6,20040416,20040416,6017.0,20040416,61,-36,6,0,2.383996,206.0


In [14]:
nevada.head(3)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,0,0,9300,10.0,C,39.036950,-121.123203,6,20070121,20070121,6057.0,20070121,-1,-74,-46,0,1.764761,76.25
1,1,1,9301,0.1,A,39.365289,-121.124769,6,20040828,20040828,6057.0,20040828,260,74,157,0,0.723467,61.00
2,2,2,9302,0.2,A,39.123450,-121.075389,6,19980809,19980809,6057.0,19980809,232,64,152,0,2.607683,99.75


In [15]:
plumas.head(3)

,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,...,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,0,0,0,0,0,0,10570,0.10,A,39.899722,...,20060811,20060811,6063.0,20060811,243,35,137,0,1.813398,99
1,1,1,1,1,1,1,10571,0.25,A,40.145833,...,20060709,20060709,6063.0,20060709,239,89,159,0,0.609016,99
2,2,2,2,2,2,2,10572,0.90,B,40.185000,...,20091108,20091108,6063.0,20091108,107,-89,-8,0,1.569229,94


In [16]:
riverside.head(3)

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,0,0,0,0,0.1,A,33.976008,-116.501361,6,20070616,20070616,6065.0,20070616,406.0,294.0,350.0,0.0,3.773714,67.077
1,1,1,1,1,1.0,B,33.735631,-117.266681,6,20031014,20031014,6065.0,20031014,356.0,211.0,283.0,0.0,1.278828,67.500
2,2,2,2,2,0.1,A,33.663369,-116.312561,6,19990202,19990202,6065.0,19990202,228.0,50.0,139.0,0.0,4.756914,60.667


In [17]:
bernardino.head(3)

,Unnamed: 0.1,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,0,13745,0.10,A,35.393300,-115.827800,6,20070810,20070810,6071.0,20070810,439,322,383,0,2.844799,58.615
1,1,13746,0.10,A,34.279722,-116.845833,6,20091113,20091113,6071.0,20091113,439,322,383,0,2.091634,109.333
2,2,13747,0.02,A,34.216389,-116.889167,6,20070812,20070812,6071.0,20070812,439,322,383,0,2.214722,67.000


In [18]:
diego.head(3)

,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,14726,0.1,A,32.661481,-116.598631,6,20041211,20041211,6073.0,20041211,239,56,150,0,3.234333,72.000
1,14727,3.0,B,32.721389,-116.557778,6,19990108,19990109,6073.0,19990108,178,50,117,0,3.375749,0.000
2,14728,0.3,B,32.657803,-116.458650,6,20050601,20050601,6073.0,20050601,250,89,172,0,3.575618,89.667


In [19]:
shasta.head(3)

,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,15990,0.1,A,40.607406,-122.496061,6,20060618,20060618,6089.0,20060618,394,217,306,0,2.997146,118.5
1,15991,0.1,A,40.716389,-121.423889,6,20070710,20070711,6089.0,20070710,394,217,306,0,1.721920,72.5
2,15992,0.5,B,40.497081,-122.360719,6,19980703,19980703,6089.0,19980703,333,161,250,0,4.217491,0.0


In [20]:
siskiyou.head(3)

,Unnamed: 0,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,START_DATE,END_DATE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,17144,1.5,B,41.574500,-122.643311,6,20020818,20020819,6093.0,20020818,317,94,206,0,1.686797,103.333
1,17145,0.2,A,41.502239,-122.737850,6,20020722,20020722,6093.0,20020722,283,139,211,0,3.466604,130.000
2,17154,0.1,A,41.719639,-122.757569,6,20020722,20020722,6093.0,20020722,283,139,211,0,3.466604,130.000


In [11]:
fire_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14151 entries, 0 to 14150
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       12181 non-null  float64
 1   FIRE_SIZE        14151 non-null  float64
 2   FIRE_SIZE_CLASS  14151 non-null  object 
 3   LATITUDE         14151 non-null  float64
 4   LONGITUDE        14151 non-null  float64
 5   STATE            14151 non-null  int64  
 6   START_DATE       14151 non-null  int64  
 7   END_DATE         14151 non-null  int64  
 8   FIPS             14151 non-null  float64
 9   DATE             14151 non-null  int64  
 10  tmax             14137 non-null  float64
 11  tmin             14137 non-null  float64
 12  tavg             14137 non-null  float64
 13  prcp             14137 non-null  float64
 14  aws              14151 non-null  float64
 15  fmc              14151 non-null  float64
 16  Unnamed: 0.3     2721 non-null   float64
 17  Unnamed: 0.2

In [16]:
fire_data.head(500)

,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,B,39.735278,-121.336389,6007.0,20080621,389.0,222.0,306.0,0.0,0.0,0.0
1,B,39.533339,-121.530711,6007.0,20010808,411.0,228.0,322.0,0.0,0.0,0.0
2,C,39.355089,-121.772239,6007.0,20041012,328.0,133.0,233.0,0.0,0.0,0.0
3,A,39.790389,-121.515500,6007.0,20041011,322.0,133.0,228.0,0.0,0.0,0.0
4,E,39.777800,-121.552470,6007.0,20080621,389.0,222.0,306.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
495,A,39.921469,-121.728597,6007.0,20070228,389.0,222.0,306.0,0.0,0.0,0.0
496,B,39.572953,-121.313275,6007.0,20070401,389.0,222.0,306.0,0.0,0.0,0.0
497,B,39.980181,-121.550731,6007.0,20040625,333.0,161.0,250.0,0.0,0.0,0.0
498,A,39.849939,-121.588689,6007.0,19980726,383.0,211.0,300.0,0.0,0.0,0.0


In [28]:
fire_data.drop(["Unnamed: 0.3"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.2"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.1"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0"], axis = 1, inplace = True)
fire_data.drop(["STATE"], axis = 1, inplace = True)
fire_data.drop(["START_DATE"], axis = 1, inplace = True)
fire_data.drop(["END_DATE"], axis = 1, inplace = True)



KeyError: ignored

In [27]:
fire_data.drop(["Unnamed: 0.4"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.5"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.6"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.7"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.8"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.9"], axis = 1, inplace = True)

In [23]:
def classify_ranges(df, column_name, ranges, classification_column):
    for range_name, range_values in ranges.items():
        df.loc[df[column_name].between(*range_values), classification_column] = range_name

# Example usage


classification_ranges = {
    'A': (0, 0.25),
    'B': (0.000000001, 9),
    'C': (9.000000001, 99),
    'D': (99.000000001, 299),
    'E': (299.000000001, 999),
    'F': (999.000000001, 4999),
    'G': (4999.000000001, 10000000)

}
  # Creating a new column to store the classifications
classify_ranges(fire_data, 'FIRE_SIZE', classification_ranges, 'FIRE_SIZE_CLASS')


In [24]:
fire_data.drop(["FIRE_SIZE"], axis = 1, inplace = True)

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(fire_data.drop(["FIRE_SIZE_CLASS"], axis=1), fire_data["FIRE_SIZE_CLASS"], test_size = .3, random_state = 101)

In [37]:
fire_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14137 entries, 0 to 14150
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FIRE_SIZE_CLASS  14137 non-null  int64  
 1   LATITUDE         14137 non-null  float64
 2   LONGITUDE        14137 non-null  float64
 3   FIPS             14137 non-null  float64
 4   DATE             14137 non-null  float64
 5   tmax             14137 non-null  float64
 6   tmin             14137 non-null  float64
 7   tavg             14137 non-null  float64
 8   prcp             14137 non-null  float64
 9   aws              14137 non-null  float64
 10  fmc              14137 non-null  float64
dtypes: float64(10), int64(1)
memory usage: 1.3 MB


In [38]:
fire_data.dropna(inplace = True)
fire_data['FIRE_SIZE_CLASS'] = fire_data['FIRE_SIZE_CLASS'].replace({'A':1 , 'B':2 , 'C':3 ,'D':4 , 'E':5 , 'F':6 , 'G':7 })

In [39]:
fire_data["DATE"] = fire_data["DATE"].astype(float)

In [63]:
X_train = torch.tensor(X_train.values, dtype = torch.float)
Y_train = torch.tensor(Y_train.values, dtype = torch.long)
X_test = torch.tensor(X_test.values, dtype = torch.float)
Y_test = torch.tensor(Y_test.values, dtype = torch.long)

In [69]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.hidden1 = nn.Linear(X_train.shape[1], 16)
    self.hidden2 = nn.Linear(16, 32)
    self.hidden3 = nn.Linear(32, 1)

  def forward(self, x):
    x = torch.relu(self.hidden1(x))
    x = torch.relu(self.hidden2(x))
    x = self.hidden3(x)
    return x

In [70]:
model = NeuralNetwork()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), .001)
print(model.parameters())

<generator object Module.parameters at 0x78516148e880>


In [71]:
Y_train = torch.flatten(Y_train)
Y_test = torch.flatten(Y_test)

In [72]:
train_data = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_data, batch_size = 32)

In [73]:
epochs = 80
loss_list = np.zeros((epochs,))

for epoch in range(epochs):
  running_loss = 0
  for inputs, targets in train_loader:
    optimizer.zero_grad() # reset optimizer gradients
    outputs = model(inputs) # forward pass data through nodes and layers
    outputs = torch.flatten(outputs)
    loss = loss_fn(outputs, targets.float()) # compute loss
    loss_list[epoch] = loss.item()
    running_loss += loss.item()
    loss.backward() #backpropagation
    optimizer.step() # update weights

  if epoch%10 == 0:
    print(f"Epoch [{epoch}/10000], Loss: {running_loss/len(train_loader)}")

Epoch [0/10000], Loss: 2364.2016837772003
Epoch [10/10000], Loss: 799.9545944127659
Epoch [20/10000], Loss: 718.6889910509358
Epoch [30/10000], Loss: 448.7343239649541
Epoch [40/10000], Loss: 368.5776700811871
Epoch [50/10000], Loss: 294.7263832738844
Epoch [60/10000], Loss: 324.89576850503175
Epoch [70/10000], Loss: 282.2378053611281


In [84]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
model.eval()

with torch.no_grad():
    outputs = model(X_test)
    predictions = outputs.flatten().tolist()
    actuals = Y_test.tolist()

predictions = torch.tensor(predictions)
actuals = torch.tensor(actuals)

mse = mean_squared_error(actuals, predictions)
r2 = r2_score(actuals, predictions)
f1 = f1_score(actuals, predictions)
rmse = np.sqrt(mse)

print('Test MSE:', mse)
print("Test r2:", r2)
print('Test RMSE:', rmse)
print('Test F1:', f1)

ValueError: ignored

In [82]:
predictions

tensor([383.2095, 382.2759, 383.8111,  ..., 384.2095, 382.8853, 383.6158])

In [83]:
Y_test

tensor([4, 2, 2,  ..., 2, 2, 2])

In [81]:
print(classification_report(predictions, Y_test))

ValueError: ignored

In [57]:
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
fire_data.groupby("FIRE_SIZE_CLASS").count()

,LATITUDE,LONGITUDE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
FIRE_SIZE_CLASS,,,,,,,,,,
2,13126,13126,13126,13126,13126,13126,13126,13126,13126,13126
3,656,656,656,656,656,656,656,656,656,656
4,151,151,151,151,151,151,151,151,151,151
5,89,89,89,89,89,89,89,89,89,89
6,73,73,73,73,73,73,73,73,73,73
7,42,42,42,42,42,42,42,42,42,42


In [89]:
fire_data.head()

,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,2,39.735278,-121.336389,6007.0,20080621.0,389.0,222.0,306.0,0.0,0.0,0.0
1,2,39.533339,-121.530711,6007.0,20010808.0,411.0,228.0,322.0,0.0,0.0,0.0
2,3,39.355089,-121.772239,6007.0,20041012.0,328.0,133.0,233.0,0.0,0.0,0.0
3,2,39.790389,-121.515500,6007.0,20041011.0,322.0,133.0,228.0,0.0,0.0,0.0
4,5,39.777800,-121.552470,6007.0,20080621.0,389.0,222.0,306.0,0.0,0.0,0.0


In [ ]:
fire_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2707 entries, 0 to 2720
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FIRE_SIZE_CLASS  2707 non-null   int64  
 1   FIPS             2707 non-null   float64
 2   DATE             2707 non-null   int64  
 3   tmax             2707 non-null   float64
 4   tmin             2707 non-null   float64
 5   tavg             2707 non-null   float64
 6   prcp             2707 non-null   float64
 7   aws              2707 non-null   float64
 8   fmc              2707 non-null   float64
dtypes: float64(7), int64(2)
memory usage: 211.5 KB


In [47]:
x = fire_data.drop("FIRE_SIZE_CLASS", axis=1)
y = fire_data["FIRE_SIZE_CLASS"]

In [48]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.20, random_state = 101)

In [49]:
lm = LogisticRegression()
lm.fit(X_train, Y_train)

LogisticRegression()

In [50]:
predictions = lm.predict(X_test)

In [51]:
mean_squared_error(Y_test, predictions)

0.30975954738330974

In [52]:
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           2       0.93      1.00      0.96      2632
           3       0.00      0.00      0.00       120
           4       0.00      0.00      0.00        33
           5       0.00      0.00      0.00        22
           6       0.00      0.00      0.00        11
           7       0.00      0.00      0.00        10

    accuracy                           0.93      2828
   macro avg       0.16      0.17      0.16      2828
weighted avg       0.87      0.93      0.90      2828



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [55]:
from sklearn import metrics

In [53]:
mean_squared_error(Y_test, predictions)

0.30975954738330974

In [58]:
r2_score(Y_test, predictions)

-0.050778499600443006

In [59]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, Y_train)

DecisionTreeClassifier()

In [60]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [61]:
y_pred_dtc = dtc.predict(X_test)

In [62]:
print(classification_report(Y_test, y_pred_dtc))

              precision    recall  f1-score   support

           2       0.95      0.94      0.94      2632
           3       0.19      0.23      0.20       120
           4       0.13      0.15      0.14        33
           5       0.06      0.05      0.05        22
           6       0.20      0.18      0.19        11
           7       0.10      0.10      0.10        10

    accuracy                           0.89      2828
   macro avg       0.27      0.27      0.27      2828
weighted avg       0.89      0.89      0.89      2828

